# CW1 - Object Recognition using CNN
To apply machine learning alorithms to clasify the testing images into object categories. Then use a model to perform classification and report quantitative results.

Due: Monday 19th April

# Introduction

The aim is to evaluate the use of CNN's in image recognition and the affect of adding multiple layers.

# Imports 

In [1]:
import numpy as np
from numpy import load
from numpy import asarray
from numpy import save
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.feature
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D


from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler


# Load Data

CIFAR-100 dataset
Each category 
contains 500 training images and 100 testing images, which are stored in two 4D arrays.

The size of each image is fixed at 32x32x3, corresponding to height, width, and colour 
channel, respectively.

There are 100 different categories of objects
each has 500 images for training and 100 images for testing.
Split the data into train and test sections

In [42]:
images = np.load('trnImage.npy')
label_fine = np.load('trnLabel_fine.npy')
label_coarse = np.load('trnLabel_coarse.npy')

#image_index = 1 # pick a specific image
#image = images[:, :, :, image_index]

test_images = np.load('tstImage.npy')
test_label_fine = np.load('tstLabel_fine.npy')
test_label_coarse = np.load('tstLabel_coarse.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Fine Shape: {label_fine.shape}')
print(f'Train Labels Coarse Shape: {label_coarse.shape}')

Images Shape: (32, 32, 3, 50000)
Train Labels Fine Shape: (50000,)
Train Labels Coarse Shape: (50000,)


## Shuffle data to ensure not ordered

In [43]:
images = shuffle(images, random_state=0)
label_fine, label_coarse = shuffle(label_fine, label_coarse, random_state=0) #make sure the samples are not ordered


test_images = shuffle(test_images, random_state=0)
test_label_fine, test_label_coarse = shuffle(test_label_fine, test_label_coarse, random_state=0)

# CNN

## Normalise the data, for each image do a hog, add how to array, train images

In [46]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))

train_images_normalised = np.divide(train_images_reshaped, 255)
test_images_normalised = np.divide(test_images_reshaped, 255)

label_coarse_reshaped = np.expand_dims(label_coarse, axis=1)
test_label_coarse_reshaped = np.expand_dims(test_label_coarse, axis=1)



In [47]:
print((train_images_normalised.shape,label_coarse_reshaped.shape))
print((test_images_normalised.shape,test_label_coarse_reshaped.shape))


((50000, 32, 32, 3), (50000, 1))
((10000, 32, 32, 3), (10000, 1))


In [48]:
xTrn_split, xTst_split, yTrn_split, yTst_split = train_test_split(train_images_normalised, label_coarse_reshaped, test_size=0.2)

In [51]:
#Dimension of the CIFAR10 dataset
print((xTrn_split.shape,yTrn_split.shape))
print((xTst_split.shape,yTst_split.shape))
print((test_images_normalised.shape, test_label_coarse_reshaped.shape))


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 20 coarse classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 200
yTrn_split=to_categorical(yTrn_split)
yTst_split=to_categorical(yTst_split)
test_label_coarse_reshaped=to_categorical(test_label_coarse_reshaped)

#Verifying the dimension after one hot encoding
print((xTrn_split.shape,yTrn_split.shape))
print((xTst_split.shape,yTst_split.shape))
print((test_images_normalised.shape,test_label_coarse_reshaped.shape))

#cifar 10



#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

tst_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(xTrn_split)
val_generator.fit(xTst_split)
test_generator.fit(test_images_normalised)

((40000, 32, 32, 3), (40000, 1))
((10000, 32, 32, 3), (10000, 1))
((10000, 32, 32, 3), (10000, 1))
((40000, 32, 32, 3), (40000, 20))
((10000, 32, 32, 3), (10000, 20))
((10000, 32, 32, 3), (10000, 20))


## Check the Data cardinality

In [17]:
train_images_reshaped = train_images_normalised.reshape(50000, 32, 32, 1)
test_images_reshaped = test_images_normalised.reshape(10000, 32, 32, 1)

#reshaping labels
label_coarse_reshaped = np.expand_dims(label_coarse, axis=1)
test_label_coarse_reshaped = np.expand_dims(test_label_coarse, axis=1)

print("train shape " + str(train_images_reshaped.shape))
print("test shape " + str(test_images_reshaped.shape) + "\n")

print("coarse shape " + str(label_coarse_reshaped.shape))
print("fine shape " + str(label_fine.shape)+ "\n")

output_num_fine = (np.unique(label_fine).shape[0])
print("Unique Fine labels " + str(output_num_fine))

output_num_coarse = (np.unique(label_coarse).shape[0])
print("unique Coarse Labels " + str(output_num_coarse))


train shape (50000, 32, 32, 1)
test shape (10000, 32, 32, 1)

coarse shape (50000, 1)
fine shape (50000,)

Unique Fine labels 100
unique Coarse Labels 20


## Split Training data into a train and test split

## Create a Tensorflow Convolutional Neural Network on the training set using Conv2D and pooling Layers
AlexNet

https://analyticsindiamag.com/hands-on-guide-to-implementing-alexnet-with-keras-for-multi-class-image-classification/#:~:text=Implementing%20in%20Keras,it%20a%20pre%2Dtrained%20model.

https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

In [8]:

alex_net = tf.keras.models.Sequential()

#1st Layer
alex_net.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)))
alex_net.add(tf.keras.layers.BatchNormalization())
alex_net.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))

#2nd Layer
alex_net.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
alex_net.add(tf.keras.layers.BatchNormalization())
alex_net.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))

#3rd Layer
alex_net.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
alex_net.add(tf.keras.layers.BatchNormalization())

#4th Layer
alex_net.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
alex_net.add(tf.keras.layers.BatchNormalization())

#5th Layer
alex_net.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
alex_net.add(tf.keras.layers.BatchNormalization())
alex_net.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))

#Fully connected Final Layers
alex_net.add(tf.keras.layers.Flatten())

#Fully connected layer 1
alex_net.add(tf.keras.layers.Dense(4096, activation='relu'))
alex_net.add(tf.keras.layers.Dropout(0.2)) #prevent overfitting

#Fully connected layer 2
alex_net.add(tf.keras.layers.Dense(2000, activation='relu'))
alex_net.add(tf.keras.layers.Dropout(0.4))

#Fully connected layer 3
alex_net.add(tf.keras.layers.Dense(10, activation='softmax'))


alex_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

## Train and fit the model

softmax - sed as the last activation function of a neural network to normalize the output of a network
use stochastic gradient descent as optimiser

In [9]:
alex_net.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='sgd',
              metrics=['accuracy'])

In [20]:
xTrn_split, xTst_split, yTrn_split, yTst_split
#Dimension of the CIFAR10 dataset
#trainig data
print((xTrn_split.shape,yTrn_split.shape))

#training test data
print((xTst_split.shape,yTst_split.shape))

#test data
print((test_images_reshaped.shape,test_label_coarse_reshaped.shape))

print(images.shape)

((40000, 32, 32, 1), (40000, 1))
((10000, 32, 32, 1), (10000, 1))
((10000, 32, 32, 1), (10000, 1))
(32, 32, 3, 50000)


In [19]:
history = alex_net.fit(xTrn_split, yTrn_split, epochs=100, 
                    validation_data=(xTst_split, yTst_split))

Epoch 1/100


ValueError: in user code:

    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replic

##  Predict labels for the testing set and check accuracy

In [ ]:
print(alex_net.predict(test_images_reshaped))

## Plot models traing curves 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = alex_net.evaluate(test_images_reshaped,  test_label_coarse, verbose=2)

print(test_acc)

In [ ]:
score = alex_net.evaluate(test_images_reshaped, test_label_coarse, verbose=0)
print(score)